In [22]:
PRFX='prep0317_2'

# setup

In [23]:
from pathlib import Path
import pickle
import dask.dataframe as dd
import pandas as pd
import dask
print('dask.__version__', dask.__version__)
import xgboost
import datetime
def dtnow(): return datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')


SEED=101
HOME='/data/git/recsys20'
p_in=f'{HOME}/input'
p_out=f'{HOME}/output/{PRFX}'
Path(p_out).mkdir(exist_ok=True, parents=True)

cols=[
'toks',
'hshtgs',
'twtid',
'media',
'links',
'domns',
'twttyp',
'lang',
'tm',

'u1id',
'u1_fllwer_cnt',
'u1_fllwing_cnt',
'u1_vrfed',
'u1_create_tm',

'u2id',
'u2_follower_cnt',
'u2_following_cnt',
'u2_vrfed',
'u2_create_tm',

'u1_fllw_u2',
'reply_tm',
'retwt_tm',
'retwt_cmmnt_tm',
'like_tm',
]

cols_val = cols[:-4]

cols_tgt_tmstmp=[
    'reply_tm',
    'retwt_tm',
    'retwt_cmmnt_tm',
    'like_tm',
]
cols_tgt=[o.split('_tm')[0] for o in cols_tgt_tmstmp]
print(cols_tgt)

dask.__version__ 2.12.0
['reply', 'retwt', 'retwt_cmmnt', 'like']


# prepare data

In [24]:
# (rcss20) ➜  recsys20 git:(master) ✗ wc -l input/trn.tsv
#  148,075,238 data/trn.tsv
# (rcss20) ➜  recsys20 git:(master) ✗ wc -l input/val.tsv
#  15,127,684 data/val.tsv

In [25]:
trntmstmp=1584412344
datetime.datetime.fromtimestamp(trntmstmp).strftime('%Y-%m-%d %H:%M:%S')

'2020-03-17 02:32:24'

In [26]:
valtmstmp=1584405047
datetime.datetime.fromtimestamp(valtmstmp).strftime('%Y-%m-%d %H:%M:%S')

'2020-03-17 00:30:47'

In [27]:
ls $p_in -hl | grep {valtmstmp}

-rw-rw-r-- 1 ubuntu ubuntu   42K Mar 18 00:08 val_12D8CEB94F89D11D7EB95EAE9689B009_1584405047.tsv
-rw-rw-r-- 1 ubuntu ubuntu  473K Mar 17 03:03 val_1584405047_1e3.tsv
-rwxrwxrwx 1 ubuntu ubuntu  6.9G Mar  3 23:25 val_1584405047.tsv*
-rw-rw-r-- 1 ubuntu ubuntu 1020K Mar 17 20:28 val_2216D01F7B48554E4211021A46816FCF_1584405047.tsv
-rw-rw-r-- 1 ubuntu ubuntu  2.5K Mar 17 22:03 val_2E18F6F53E3CF073911AF0A93BBE5373_1584405047.tsv
-rw-rw-r-- 1 ubuntu ubuntu   26K Mar 17 23:06 val_4690215948DBF6872B8ED1C2BC87B17E_1584405047.tsv
-rw-rw-r-- 1 ubuntu ubuntu   994 Mar 17 21:01 val_515E873C86EE1577E75FA2387B7FA59E_1584405047.tsv
-rw-rw-r-- 1 ubuntu ubuntu   90K Mar 18 00:40 val_60A3DB168094D41241E45E0DE3539BC0_1584405047.tsv
-rw-rw-r-- 1 ubuntu ubuntu   74K Mar 18 01:11 val_97F81BD92A1ACA3F1F43C154E689350F_1584405047.tsv
-rw-rw-r-- 1 ubuntu ubuntu  134K Mar 18 02:12 val_A6B70CDF8C7B934D4A218CA9B6B7FDB4_1584405047.tsv
-rw-rw-r-- 1 ubuntu ubuntu  180K Mar 18 02:43 val_AA0254541959271ED3453119B787D0C

In [28]:
lang='2216D01F7B48554E4211021A46816FCF'

In [29]:
dftrn=dd.read_csv(
    f'{p_in}/trn_{lang}_{trntmstmp}.tsv',
    sep='\x01', header=None, names=cols,)

In [30]:
dfval=dd.read_csv(
    f'{p_in}/val_{lang}_{valtmstmp}.tsv',
    sep='\x01',header=None,names=cols_val,)

# colwise

In [31]:
dftrn[cols_tgt]=~dftrn[cols_tgt_tmstmp].isna().astype('int8')+2
display(dftrn[cols_tgt_tmstmp].head())
ys=dftrn[cols_tgt]
ys.head()

,reply_tm,retwt_tm,retwt_cmmnt_tm,like_tm
0,NaN,NaN,NaN,1.581224e+09
1,NaN,1.581082e+09,NaN,NaN
2,NaN,NaN,NaN,1.581337e+09
3,NaN,1.581099e+09,NaN,1.581099e+09
4,NaN,NaN,NaN,1.581058e+09


,reply,retwt,retwt_cmmnt,like
0,0,0,0,1
1,0,1,0,0
2,0,0,0,1
3,0,1,0,1
4,0,0,0,1


In [32]:
def prepcols(df, isval=True):
    df['len_toks']=df.toks.apply(len, meta=('toks', 'int64'))
    for media in ['Photo', 'Video', 'GIF']:
        df[f'has_media_{media}'] = df.media.fillna('').apply(lambda x: media in x, 
                                                                     meta=('media', 'string'))
    for col in ['hshtgs', 'links', 'domns',]:
        df[f'num_{col}'] = df[col].fillna('').apply(lambda x: len(x.split('\t')) if len(x) else 0,
                                                          meta=(col, 'string'))
    df['tmdlta_u2u1']  = df.u2_create_tm - df.u1_create_tm
    df['tmdlta_twtu1'] = df.tm - df.u1_create_tm
    df['tmdlta_twtu2'] = df.tm - df.u2_create_tm
    cols_num = [
       'tm', 'u1_fllwer_cnt', 'u1_fllwing_cnt', 'u1_vrfed',
       'u1_create_tm', 'u2_follower_cnt', 'u2_following_cnt',
       'u2_vrfed', 'u2_create_tm', 'u1_fllw_u2', 'len_toks', 'has_media_Photo',
       'has_media_Video', 'has_media_GIF', 'num_hshtgs', 'num_links',
       'num_domns', 'tmdlta_u2u1', 'tmdlta_twtu1', 'tmdlta_twtu2']
    cols_cat0 = ['twttyp',]
    df_cat = dd.get_dummies(df[cols_cat0].categorize())
    cols_cat = sorted(df_cat.columns)
    df_cat=df_cat[cols_cat]
    df = dd.concat([df,df_cat], axis=1)
    cols_feat = cols_num+cols_cat
    if isval: return df, cols_feat
    
    df[cols_tgt]=~df[cols_tgt_tmstmp].isna().astype('int8')+2
    return df, cols_feat

In [33]:
dftrn=dd.read_csv(
    f'{p_in}/trn_{lang}_{trntmstmp}.tsv',
    sep='\x01', header=None, names=cols,)

dfval=dd.read_csv(
    f'{p_in}/val_{lang}_{valtmstmp}.tsv',
    sep='\x01',header=None,names=cols_val,)

dftrn, cols_feat = prepcols(dftrn, isval=False)
dfval, cols_feat = prepcols(dfval, isval=True)

/data/anaconda3/envs/rcss20/lib/python3.7/site-packages/dask/dataframe/multi.py:1056: UserWarning: Concatenating dataframes with unknown divisions.
We're assuming that the indexes of each dataframes are 
 aligned. This assumption is not generally safe.
  "Concatenating dataframes with unknown divisions.\n"


In [34]:
dftrn.head()

,toks,hshtgs,twtid,media,links,domns,twttyp,lang,tm,u1id,...,tmdlta_u2u1,tmdlta_twtu1,tmdlta_twtu2,twttyp_Quote,twttyp_Retweet,twttyp_TopLevel,reply,retwt,retwt_cmmnt,like
0,101\t1367\t43877\t20854\t27921\t39578\t1364\t3...,NaN,A4301795BF91376D36101DE9BF1EF530,NaN,NaN,NaN,TopLevel,2216D01F7B48554E4211021A46816FCF,1581152619,26A84E513107F8CCA3FDD38DE56A7AF4,...,-24265405,33240151,57505556,0,0,1,0,0,0,1
1,101\t56898\t137\t169\t168\t182\t168\t177\t168\...,NaN,19B317C489098848A19C90DA4B23CFE2,NaN,NaN,NaN,Retweet,2216D01F7B48554E4211021A46816FCF,1581075564,23AAC02F6EDF248A2E6EB2140BCB9516,...,108239404,239531391,131291987,0,1,0,0,1,0,0
2,101\t1353\t44728\t1353\t23948\t23290\t18384\t4...,NaN,73D91129A17356C8F899343BD5B8801E,NaN,NaN,NaN,TopLevel,2216D01F7B48554E4211021A46816FCF,1581336525,A39F8CD1AF10515208543F6897843B5F,...,-260416547,69013052,329429599,0,0,1,0,0,0,1
3,101\t1357\t23290\t34590\t17896\t47357\t1343\t6...,NaN,F9C9646448986252BCC80EAB7439075E,NaN,NaN,NaN,TopLevel,2216D01F7B48554E4211021A46816FCF,1581080654,4645A47F024F5DB1B0739CD300918D1E,...,161607271,276109615,114502344,0,0,1,0,1,0,1
4,101\t1339\t23948\t75301\t18384\t89291\t49812\t...,30BB2727F7A5665A40430ED55987910F,5BE97362AA81B5A95C7A99606BB96AEB,NaN,NaN,NaN,TopLevel,2216D01F7B48554E4211021A46816FCF,1581057914,C2665B06F7CA141C18240EBB15A738A6,...,-243695622,72450037,316145659,0,0,1,0,0,0,1


In [35]:
dfval.head()

,toks,hshtgs,twtid,media,links,domns,twttyp,lang,tm,u1id,...,has_media_GIF,num_hshtgs,num_links,num_domns,tmdlta_u2u1,tmdlta_twtu1,tmdlta_twtu2,twttyp_Quote,twttyp_Retweet,twttyp_TopLevel
0,101\t56898\t137\t15444\t35451\t11211\t11396\t1...,NaN,65B61A3E5EE99D02AAFF70153F9B3EC7,Video,NaN,NaN,Retweet,2216D01F7B48554E4211021A46816FCF,1582008933,2EA151284906793673D663C4299C292F,...,False,0,0,0,-218958162,118000563,336958725,0,1,0
1,101\t1321\t74201\t18395\t89291\t111395\t75932\...,43210E6DD8A82AEA67EC41BAFAC933CB\tA240BB5A5111...,0BB4BB026C684FCA5251420E99513B5F,Photo,5D2C8198CB3288A42B1ACDECF668B548,E91CDEC8DC7ABF30592FA024616FF970,TopLevel,2216D01F7B48554E4211021A46816FCF,1581666064,0A6EFA2C64BFB91416D332B013D97045,...,False,4,1,1,-77754743,64872013,142626756,0,0,1
2,101\t56898\t137\t35060\t19986\t93382\t10206\t1...,NaN,5B743EDB0607771835FDB815FE81F69D,NaN,NaN,NaN,Retweet,2216D01F7B48554E4211021A46816FCF,1581863748,7D27052B87A7B75876561CBFDA648CF6,...,False,0,0,0,-112887550,51049650,163937200,0,1,0
3,101\t56898\t137\t26064\t13321\t10731\t21442\t1...,NaN,83D2C263990F731DADE25820724ED2EF,NaN,NaN,NaN,Retweet,2216D01F7B48554E4211021A46816FCF,1581788107,E8C98FBE8AA8D4447F0CDF1393095109,...,False,0,0,0,-35955765,127905794,163861559,0,1,0
4,101\t56898\t137\t10846\t53543\t75268\t46382\t1...,NaN,93F971E797EAF55649CC6C366D571E20,NaN,NaN,NaN,Retweet,2216D01F7B48554E4211021A46816FCF,1582102168,34C142E75A93212BE7310CD0095B5F87,...,False,0,0,0,-104016159,60159461,164175620,0,1,0


# count

In [36]:
coltgt=cols_tgt[0]
dfpos=dftrn[dftrn[coltgt]==1]
dfpos.head()

,toks,hshtgs,twtid,media,links,domns,twttyp,lang,tm,u1id,...,tmdlta_u2u1,tmdlta_twtu1,tmdlta_twtu2,twttyp_Quote,twttyp_Retweet,twttyp_TopLevel,reply,retwt,retwt_cmmnt,like
5,101\t1334\t90865\t73145\t1357\t55898\t1365\t11...,NaN,ECE7B3DCCA5FE7DF521356EDAF319247,NaN,NaN,NaN,TopLevel,2216D01F7B48554E4211021A46816FCF,1581088856,ED4A0CEF7AAAC810A8F910B96A5F1EDE,...,-184287785,82067832,266355617,0,0,1,1,0,0,1
13,101\t31558\t1321\t104014\t1322\t18395\t79591\t...,NaN,A048A57F776EB7976501144C4E026F23,NaN,NaN,NaN,TopLevel,2216D01F7B48554E4211021A46816FCF,1581188191,E8A742121E6EFBAF513E51DDDC5BE24B,...,-32929716,207741364,240671080,0,0,1,1,0,0,0
33,101\t56326\t16832\t1334\t111402\t75301\t27921\...,NaN,69770BAC3CF9196C7B688ADC7323E830,NaN,NaN,NaN,TopLevel,2216D01F7B48554E4211021A46816FCF,1581409695,177ABCFB2E917837F9F6C81C05D70D9C,...,21447607,90034089,68586482,0,0,1,1,0,0,1
106,101\t1354\t106656\t1322\t36089\t29400\t15080\t...,NaN,B0B2CE897A5385257F2F621607F4FBA8,NaN,NaN,NaN,TopLevel,2216D01F7B48554E4211021A46816FCF,1581352923,4CA923AE4E25C5C35FC47B2ED5C812CD,...,7502511,308721574,301219063,0,0,1,1,0,0,1
158,101\t1321\t22945\t23290\t24458\t54885\t31538\t...,NaN,6E2E702FA1B708D6E071E2A67E07D882,NaN,NaN,NaN,TopLevel,2216D01F7B48554E4211021A46816FCF,1581044512,191614E9D775EE6314F78191AFBFAF7B,...,195369937,221253185,25883248,0,0,1,1,0,0,1


In [37]:
dfcnt=dfpos.groupby(['u1id', 'u2id']).size().reset_index()
dfcnt.columns=['u1id','u2id',f'{coltgt}_cnt']
dfcnt.head()

,u1id,u2id,reply_cnt
0,00775C5A013771CF74509F9985520AAE,2E347E868D6E870244DB8B1A2F3BEB9B,1
1,00775C5A013771CF74509F9985520AAE,43D5AD5B9707E194F685BBF88D476833,1
2,00775C5A013771CF74509F9985520AAE,ABD74A48C398A14D65176ADAAA7B110E,1
3,00775C5A013771CF74509F9985520AAE,C5B2AD1C69D8EC327646A9F63B79432C,1
4,03841B56A56065D3E51FBC939F56B87F,4F6926A36DA8CBEB6BC8E11C9DA9F999,1


In [38]:
# dftrn = dd.merge(dftrn,dfcnt,how='left',left_on=['u1id','u2id'],right_on=['u1id', 'u2id'])
# dftrn[f'{coltgt}_cnt'] = dftrn[f'{coltgt}_cnt'].fillna(0)

In [39]:
dfval = dd.merge(dfval,dfcnt,how='left',left_on=['u1id','u2id'],right_on=['u1id', 'u2id'])
dfval[f'{coltgt}_cnt'] = dfval[f'{coltgt}_cnt'].fillna(0)

In [40]:
dfval.head()

,toks,hshtgs,twtid,media,links,domns,twttyp,lang,tm,u1id,...,num_hshtgs,num_links,num_domns,tmdlta_u2u1,tmdlta_twtu1,tmdlta_twtu2,twttyp_Quote,twttyp_Retweet,twttyp_TopLevel,reply_cnt
0,101\t56898\t137\t15444\t35451\t11211\t11396\t1...,NaN,65B61A3E5EE99D02AAFF70153F9B3EC7,Video,NaN,NaN,Retweet,2216D01F7B48554E4211021A46816FCF,1582008933,2EA151284906793673D663C4299C292F,...,0,0,0,-218958162,118000563,336958725,0,1,0,0.0
1,101\t1321\t74201\t18395\t89291\t111395\t75932\...,43210E6DD8A82AEA67EC41BAFAC933CB\tA240BB5A5111...,0BB4BB026C684FCA5251420E99513B5F,Photo,5D2C8198CB3288A42B1ACDECF668B548,E91CDEC8DC7ABF30592FA024616FF970,TopLevel,2216D01F7B48554E4211021A46816FCF,1581666064,0A6EFA2C64BFB91416D332B013D97045,...,4,1,1,-77754743,64872013,142626756,0,0,1,0.0
2,101\t56898\t137\t35060\t19986\t93382\t10206\t1...,NaN,5B743EDB0607771835FDB815FE81F69D,NaN,NaN,NaN,Retweet,2216D01F7B48554E4211021A46816FCF,1581863748,7D27052B87A7B75876561CBFDA648CF6,...,0,0,0,-112887550,51049650,163937200,0,1,0,0.0
3,101\t56898\t137\t26064\t13321\t10731\t21442\t1...,NaN,83D2C263990F731DADE25820724ED2EF,NaN,NaN,NaN,Retweet,2216D01F7B48554E4211021A46816FCF,1581788107,E8C98FBE8AA8D4447F0CDF1393095109,...,0,0,0,-35955765,127905794,163861559,0,1,0,0.0
4,101\t56898\t137\t10846\t53543\t75268\t46382\t1...,NaN,93F971E797EAF55649CC6C366D571E20,NaN,NaN,NaN,Retweet,2216D01F7B48554E4211021A46816FCF,1582102168,34C142E75A93212BE7310CD0095B5F87,...,0,0,0,-104016159,60159461,164175620,0,1,0,0.0


# combine steps

In [55]:
dftrn=dd.read_csv(
    f'{p_in}/trn_{lang}_{trntmstmp}.tsv',
    sep='\x01', header=None, names=cols,)

dfval=dd.read_csv(
    f'{p_in}/val_{lang}_{valtmstmp}.tsv',
    sep='\x01',header=None,names=cols_val,)

dftrn, cols_feat = prepcols(dftrn, isval=False)
dfval, cols_feat = prepcols(dfval, isval=True)

def prepcnts(dftrn,dfval):
    for coltgt in cols_tgt:
        dfpos=dftrn[dftrn[coltgt]==1]
        dfcnt=dfpos.groupby(['u1id', 'u2id']).size().reset_index()
        dfcnt.columns=['u1id','u2id',f'{coltgt}_cnt']
        dfval = dd.merge(dfval,dfcnt,how='left',left_on=['u1id','u2id'],right_on=['u1id', 'u2id'])
        dfval[f'{coltgt}_cnt'] = dfval[f'{coltgt}_cnt'].fillna(0)
    cols_cnt = [f'{coltgt}_cnt' for coltgt in cols_tgt]
    return dfval, cols_cnt

/data/anaconda3/envs/rcss20/lib/python3.7/site-packages/dask/dataframe/multi.py:1056: UserWarning: Concatenating dataframes with unknown divisions.
We're assuming that the indexes of each dataframes are 
 aligned. This assumption is not generally safe.
  "Concatenating dataframes with unknown divisions.\n"


In [56]:
def prep_lang(lang):
    dftrn=dd.read_csv(
    f'{p_in}/trn_{lang}_{trntmstmp}.tsv',
    sep='\x01', header=None, names=cols,)

    dfval=dd.read_csv(
        f'{p_in}/val_{lang}_{valtmstmp}.tsv',
        sep='\x01',header=None,names=cols_val,)

    dftrn, cols_feat_trn = prepcols(dftrn, isval=False)
    dfval, cols_feat_val = prepcols(dfval, isval=True)
    assert cols_feat_trn==cols_feat_val
    dfval, cols_cnt=prepcnts(dftrn,dfval)
    return dftrn, dfval, cols_feat, cols_cnt

In [57]:
dftrn, dfval, cols_feat, cols_cnt = prep_lang(lang)

In [58]:
pickle.dump(cols_feat, open(f'{p_out}/cols_feat.p','wb'))

In [59]:
pickle.dump(cols_cnt, open(f'{p_out}/cols_cnt.p','wb'))

In [60]:
%%time
dftrn.to_csv(f'{p_out}/dftrn_{lang}_{trntmstmp}__{PRFX}.tsv',sep='\x01',index=False,single_file=True)

CPU times: user 711 ms, sys: 73 ms, total: 784 ms
Wall time: 789 ms


['/data/git/recsys20/output/prep0317_2/dftrn_2216D01F7B48554E4211021A46816FCF_1584412344__prep0317_2.tsv']

In [61]:
%%time
dfval.to_csv(f'{p_out}/dfval_{lang}_{valtmstmp}__{PRFX}.tsv',sep='\x01',index=False,single_file=True)

CPU times: user 525 ms, sys: 44.5 ms, total: 570 ms
Wall time: 490 ms


['/data/git/recsys20/output/prep0317_2/dfval_2216D01F7B48554E4211021A46816FCF_1584405047__prep0317_2.tsv']